### Connect your GPU to the code.

Watch this video to get basic knowledge [Youtube](https://youtu.be/YmDaqXMIoeY?si=JdFTtz_G7s9Ni9In.).

Follow the Youtube Video for around 3 minutes uptill the datasets part.

For the documantation follow This [Link](https://shawnhymel.com/1961/how-to-install-tensorflow-with-gpu-support-on-windows/).

### Add the required folders before running

From The VGLC GitHub copy the processed folder.
https://github.com/theVGLC/theVGLC

Follow The folder structure: Super Mario Bros/Processed

Create the `Processed` folder in the same directory as the `MarioGAN.ipynb` and add the above .txt files in this folder.

Run the code below.

In [2]:
import numpy as np
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Reshape, Flatten, Input, BatchNormalization, LeakyReLU, UpSampling2D, Conv2D
from keras.optimizers import Adam
import os

# Set dimensions
latent_dim = 100
max_height, max_width = 16, 372  # Adjust to match your dataset
level_shape = (max_height, max_width, 1)

# Create character mappings from directory
def create_char_mappings_from_directory(directory_path):
    char_set = set()

    for file_name in os.listdir(directory_path):
        file_path = os.path.join(directory_path, file_name)

        # Skip if the file_path is a directory
        if os.path.isdir(file_path):
            continue

        with open(file_path, 'r') as file:
            for line in file:
                char_set.update(line.strip())

    char_to_int = {char: idx for idx, char in enumerate(sorted(char_set))}
    int_to_char = {idx: char for char, idx in char_to_int.items()}

    return char_to_int, int_to_char

# Preprocess levels
def preprocess_levels(directory_path, max_width, max_height, char_to_int):
    levels = []

    for file_name in os.listdir(directory_path):
        file_path = os.path.join(directory_path, file_name)

        # Skip if the file_path is a directory
        if os.path.isdir(file_path):
            continue

        level = []
        with open(file_path, 'r') as file:
            for line in file:
                row = [char_to_int.get(char, -1) for char in line.strip()]
                # Pad or truncate rows to max_width
                if len(row) < max_width:
                    row.extend([-1] * (max_width - len(row)))
                else:
                    row = row[:max_width]
                level.append(row)

        # Pad or truncate levels to max_height
        if len(level) < max_height:
            level.extend([[-1] * max_width] * (max_height - len(level)))
        else:
            level = level[:max_height]

        levels.append(np.array(level))

    return np.array(levels)


# Build generator
def build_generator(latent_dim):
    model = Sequential()
    model.add(Dense(128 * (max_height // 4) * (max_width // 4), activation="relu", input_dim=latent_dim))
    model.add(Reshape((max_height // 4, max_width // 4, 128)))

    # Upsample to (max_height // 2, max_width // 2, 128)
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2D(128, kernel_size=3, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    # Upsample to (max_height, max_width, 128)
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2D(64, kernel_size=3, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    # Final layer to match the shape of the output
    model.add(Conv2D(1, kernel_size=3, padding="same", activation='tanh'))
    return model

# Build discriminator
def build_discriminator(level_shape):
    model = Sequential()
    model.add(Input(shape=level_shape))

    model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    return model

# Compile the GAN
def compile_gan(generator, discriminator):
    discriminator.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5), metrics=['accuracy'])
    discriminator.trainable = False

    gan_input = Input(shape=(latent_dim,))
    generated_level = generator(gan_input)
    gan_output = discriminator(generated_level)

    gan = Model(gan_input, gan_output)
    gan.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))
    return gan

# Save generated levels to file
def save_generated_levels(generator, epoch, latent_dim=100):
    noise = np.random.normal(0, 1, (1, latent_dim))
    gen_levels = generator.predict(noise)
    gen_levels = np.squeeze(gen_levels)  # Remove single-dimensional entries
    gen_levels_char = convert_to_char(gen_levels, int_to_char)
    save_level_to_file(gen_levels_char, f"./generated_level_{epoch}.txt")

def save_level_to_file(level, file_path):
    with open(file_path, 'w') as f:
        for row in level:
            f.write(''.join(row) + '\n')
    print(f"Generated level saved to {file_path}")

# Convert numeric levels to characters
def convert_to_char(numeric_level, int_to_char):
    return [[int_to_char.get(int(val), '?') for val in row] for row in numeric_level]

# Train the GAN
def train_gan(generator, discriminator, gan, data, epochs, batch_size, latent_dim):
    half_batch = batch_size // 2
    for epoch in range(epochs):
        # Train discriminator
        idx = np.random.randint(0, data.shape[0], half_batch)
        real_levels = data[idx]
        real_labels = np.ones((half_batch, 1))
        noise = np.random.normal(0, 1, (half_batch, latent_dim))
        generated_levels = generator.predict(noise)
        fake_labels = np.zeros((half_batch, 1))
        d_loss_real = discriminator.train_on_batch(real_levels, real_labels)
        d_loss_fake = discriminator.train_on_batch(generated_levels, fake_labels)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train generator
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        valid_y = np.ones((batch_size, 1))
        g_loss = gan.train_on_batch(noise, valid_y)

        # Print the progress
        print(f"{epoch} [D loss: {d_loss[0]}, acc.: {d_loss[1]*100}%] [G loss: {g_loss}]")

        # If at save interval => save generated level samples
        if epoch % 100 == 0:
            save_generated_levels(generator, epoch)

# Main execution
directory_path = './Processed'
char_to_int, int_to_char = create_char_mappings_from_directory(directory_path)
num_classes = len(char_to_int)

# Convert numeric levels to the format needed for the GAN
numeric_levels = preprocess_levels(directory_path, max_width, max_height, char_to_int)
numeric_levels = numeric_levels.reshape(-1, max_height, max_width, 1)

# Normalize data to [-1, 1]
numeric_levels = (numeric_levels.astype(np.float32) - num_classes / 2) / (num_classes / 2)

# Create and compile models
generator = build_generator(latent_dim)
discriminator = build_discriminator(level_shape)
gan = compile_gan(generator, discriminator)

# Train the GAN
train_gan(generator, discriminator, gan, numeric_levels, epochs=10000, batch_size=64, latent_dim=latent_dim)


<>:161: SyntaxWarning: invalid escape sequence '\T'
<>:161: SyntaxWarning: invalid escape sequence '\T'
C:\Users\Dell\AppData\Local\Temp\ipykernel_3372\48845432.py:161: SyntaxWarning: invalid escape sequence '\T'
  directory_path = '.\TheVGLC\Super Mario Bros\Processed'
d:\Deep Learning\myenv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
d:\Deep Learning\myenv\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 733ms/step


d:\Deep Learning\myenv\Lib\site-packages\keras\src\backend\tensorflow\trainer.py:75: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


0 [D loss: 0.6728358268737793, acc.: 82.03125%] [G loss: [array(0.6797489, dtype=float32), array(0.6797489, dtype=float32), array(0.640625, dtype=float32)]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 259ms/step
Generated level saved to ./generated_level_0.txt
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 266ms/step
1 [D loss: 0.6779319047927856, acc.: 68.48958730697632%] [G loss: [array(0.6803805, dtype=float32), array(0.6803805, dtype=float32), array(0.609375, dtype=float32)]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 302ms/step
2 [D loss: 0.678724467754364, acc.: 63.28125%] [G loss: [array(0.6803109, dtype=float32), array(0.6803109, dtype=float32), array(0.578125, dtype=float32)]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 247ms/step
3 [D loss: 0.6801572442054749, acc.: 60.04464626312256%] [G loss: [array(0.6813884, dtype=float32), array(0.6813884, dtype=float32), array(0.5625, dtype=float32)]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 244ms/step
4 [D loss: 0.6805479526519775, acc.: 58.05555582046509%] [G loss: [array(0.68173754, dtype=float32), array(0.6817

C:\Users\Dell\AppData\Local\Temp\ipykernel_3372\48845432.py:161: SyntaxWarning: invalid escape sequence '\T'
  directory_path = '.\TheVGLC\Super Mario Bros\Processed'


DecodeError: Error parsing message